In [6]:
import pandas as pd
import numpy as np
from utility.util import clean_columns

from matplotlib import pyplot as plt
from utility.util import plot_gender_fraction_over_time

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import os
import string
import zipfile
import shutil

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from IPython.display import clear_output

from ipywidgets import IntSlider, Output
from IPython.display import display

In [7]:
qR = pd.read_csv("./utility/data/Lab 1 Quiz.csv")
# qR.head()

In [8]:
iCare = qR[['question_number','question_title','answers','question_content']]
# iCare.iloc[8]['question_content']
pd.set_option("display.max_colwidth", -1)
# iCare[:20]

c:\users\thorm\miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
def cleaning_up(df):
    new_df = df
    for i in new_df.index:
#       changing the question content
        answers = new_df.iloc[i]['answers'].replace("0","").translate(str.maketrans("", "", string.punctuation))
        q_content = new_df.iloc[i]['question_content'].replace("0","").translate(str.maketrans("", "", string.punctuation))
        new_df.at[i, 'answers'] = answers
        new_df.at[i, 'question_content'] = q_content
#       changing the answer content
        if "radioinput" in new_df.at[i, 'question_content'] or "selectallinput" in new_df.at[i, 'question_content']:
            new_df.at[i, 'question_content'] = "multi_choice"
            continue;
        elif "text" in new_df.at[i, 'question_content']:
            new_df.at[i, 'question_content'] = "text"
            continue;
    return new_df

clean = cleaning_up(iCare)
q_num_cont = clean[["question_number", "question_content"]]
# clean[:15]
# %timeit cleaning_up_faster(iCare)


In [10]:
def short_answer (df, q_num):
    obj = SentimentIntensityAnalyzer()
    rows = []
    count = 0
    for i in df.index:
        if df.iloc[i]['question_number'] == q_num:
            count += 1
            ans = str(df.iloc[i]['answers'])
            num_words = len(ans.split())
            sent_score = obj.polarity_scores(ans)
            if num_words != 0 and num_words != 1 and num_words != 2:
                rows.append([sent_score['neg'], sent_score['neu'], sent_score['pos'], sent_score['compound'], ans])
#                 normal_sent_score = sent_score['compound']/num_words
#                 rows.append([normal_sent_score, ans])
    scores = pd.DataFrame(rows, columns=["sentiment_score_neg","sentiment_score_neu","sentiment_score_pos","sentiment_score_comp", "response"])
#     scores = pd.DataFrame(rows, columns=["sentiment_score_comp", "response"])
    scores.sort_values(by=['sentiment_score_comp'], inplace=True, ascending=False)
    scores = scores.drop(["sentiment_score_neg","sentiment_score_neu","sentiment_score_pos"], axis = 1)
    scores = scores.reset_index()
    index = scores.index
    numrows = len(index)
#     print(numrows)
#     The commented secionts above change to a normalized compound score
#     Begin buttons
    global ind_1
    global ind_2
    ind_1 = -5
    ind_2 = 0
    out = Output()
    btn = widgets.Button(description='>')
    btn2 = widgets.Button(description='<')
    display(widgets.HBox((btn2, btn)))
    display(out)
    def init():
        display(scores)
        clear_output(wait=True)
    def add(obj):
        global ind_1
        global ind_2
        if ind_2 > (int(len(scores.index))-5):
            return
        ind_1 +=5
        ind_2+=5
        
        with out:
            display(scores[ind_1:ind_2])
            clear_output(wait=True)
    def subtract(obj):
        global ind_1
        global ind_2
        if ind_1 < 4:
            return
        ind_1 -=5
        ind_2 -=5
        with out:
            display(scores[ind_1:ind_2])
            clear_output(wait=True)

    # display(IntSlider())
  
    btn.object_position = 'right'
    btn2.object_position = 'left'
    btn.on_click(add)
    btn2.on_click(subtract)

# short_answer(clean, 5.4)

In [11]:
def get_question_nums(df):
    count = 0
    arr = []
    first_num = df.iloc[0]['question_number']
    for index,row in df.iterrows():
        arr.append((f"{df.iloc[index]['question_number']}) {df.iloc[index]['question_title']}",df.iloc[index]['question_number']))
        count += 1
        index = index+1
        if(df.iloc[index]['question_number'] == first_num):
            return arr
            break                 
            
# get_question_nums(clean)

In [12]:
def multi_choice_input(df, q_num):
    answers = {}
    count = 0
    for i in df.index:
        if df.iloc[i]['question_number'] == q_num:
            count+=1
            if df.iloc[i]['answers'] in answers:
                answers[df.iloc[i]['answers']] += 1
            elif df.iloc[i]['answers'] not in answers:
                answers[df.iloc[i]['answers']] = 1

    #visual portion
    lists = sorted(answers.items())
    x, y = zip(*lists)
    plt.bar(x, y)
    plt.xticks(rotation = 35, ha = 'right')
    plt.show()
    #numbers portion
    for ans in answers.keys():
        percent = np.round((answers[ans]/count)*100,2)
        print(f'{percent}% of people chose \"{ans}\"')

In [13]:
def get_anything (df, q_num):
    pd.set_option('display.max_colwidth', None)
    q_name = df.loc[df['question_number'] == q_num, 'question_title'].iloc[0]
    print(f'{q_num}. {q_name}')
    for i in df.index:
        if df.iloc[i]["question_number"] == q_num:
            if df.iloc[i]["question_content"] == "multichoice":
                multi_choice_input(df, q_num)
                break
            elif df.iloc[i]["question_content"] == "text":
                return short_answer(df, q_num)
                break
# ABC ex 5.1
# text ex 1.0
# select ex 2.1
# radio ex 4.1
# clean is the cleaned dataframe
# get_anything(clean, 5.4)

In [14]:
arr = get_question_nums(clean)
widgets.interact(get_anything, df=fixed(clean), q_num=arr, layout={'width': 'initial'})

interactive(children=(Dropdown(description='q_num', options=(("1.0) What is the one thing you learned from tod…

<function __main__.get_anything(df, q_num)>